** This file gives a brief overview of the capabilities of the code. **

* The codes fit red arm spectra from Magellan/MIKE. 

* Since the codes are meant to be clean and minimal, it should not be hard to tweak the source codes for other purposes (say fitting the blue arm spectrum).

* The code fit for basic stellar parameters (Teff, logg, [Fe/H], [$\alpha$/Fe]), the broadening $v_{\rm broad}$, radial velocity, and continuum, to all spectral orders, simultaneously.

In [1]:
%matplotlib inline

# import packages
import numpy as np
from scipy.optimize import curve_fit
from scipy import interpolate
from scipy import signal
from scipy.stats import norm

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import gridspec

from cycler import cycler

# import The Payne (https://github.com/tingyuansen/The_Payne)
from Payne4MIKE import spectral_model
from Payne4MIKE import utils
from Payne4MIKE import fitting
from Payne4MIKE.read_spectrum import read_carpy_fits


In [2]:
# define plot properties
import matplotlib.cm as cm

from matplotlib import rcParams
from matplotlib import rc
from mpl_toolkits.axes_grid1 import make_axes_locatable

def rgb(r,g,b):
    return (float(r)/256.,float(g)/256.,float(b)/256.)

cb2 = [rgb(31,120,180), rgb(255,127,0), rgb(51,160,44), rgb(227,26,28), \
       rgb(10,10,10), rgb(253,191,111), rgb(178,223,138), rgb(251,154,153)]

rcParams['figure.figsize'] = (11,7.5)
rcParams['figure.dpi'] = 300

rcParams['lines.linewidth'] = 1

rcParams['axes.prop_cycle'] = cycler('color', cb2)
rcParams['axes.facecolor'] = 'white'
rcParams['axes.grid'] = False

rcParams['patch.facecolor'] = cb2[0]
rcParams['patch.edgecolor'] = 'white'

rcParams['font.family'] = 'Bitstream Vera Sans' 
rcParams['font.size'] = 25
rcParams['font.weight'] = 300


> Restore The Payne emulator.

In [3]:
# assuming Kurucz models
NN_coeffs, wavelength_payne = utils.read_in_neural_network()
w_array_0, w_array_1, w_array_2, b_array_0, b_array_1, b_array_2, x_min, x_max = NN_coeffs


> Read in MIKE spectra.

In [4]:
# an example of a MIKE spectrum to be fitted
wavelength, spectrum, spectrum_err = utils.read_in_example()

# or restore your own here
#read_path =  "star-2red_multi.fits"
#wavelength, spectrum, spectrum_err = read_carpy_fits(read_path)

#-----------------------------------------------------------------------------------
# restore a default hot star spectrum to determine telluric features
wavelength_blaze, spectrum_blaze, spectrum_err_blaze = utils.read_in_blaze_spectrum()

# or restore your own here
#read_path =  "Hot_Star_HR9087.fits"
#wavelength_blaze, spectrum_blaze, spectrum_err_blaze = read_carpy_fits(read_path)



/Users/yting/anaconda3/lib/python3.7/site-packages/Payne4MIKE-0.1-py3.7.egg/Payne4MIKE/read_spectrum.py:98: RuntimeWarning: divide by zero encountered in power
  ivar = image[0].data[noise_ext]**(-2)


> Massaging the spectra into a digestable format.

In [5]:
# cull nonsensible values
spectrum = np.abs(spectrum)
spectrum_blaze = np.abs(spectrum_blaze)

# rescale the spectra by its median so it has a more reasonable range
spectrum, spectrum_err = utils.scale_spectrum_by_median(spectrum, spectrum_err)
spectrum_blaze, spectrum_err_blaze = utils.scale_spectrum_by_median(spectrum_blaze, spectrum_err_blaze)

# truncate the end of each orders as they can contain nonsensible values
spectrum = spectrum[:,30:-15]
spectrum_err = spectrum_err[:,30:-15]
wavelength = wavelength[:,30:-15]

# match the wavelength (blaze -> spectrum)
spectrum_blaze, wavelength_blaze = utils.match_blaze_to_spectrum(wavelength, spectrum, wavelength_blaze, spectrum_blaze)

# use the blaze to determine telluric region
spectrum_err = utils.mask_telluric_region(spectrum_err, spectrum_blaze)


> Fit the spectrum.

In [6]:
# the range of RV that we will search (in the unit of 100 km/s)
# expand/refine the range of RV if the fit is stuck in a local minimum
RV_array=np.linspace(-1,1.,6)

popt_best, model_spec_best, chi_square = fitting.fitting_mike(spectrum, spectrum_err, spectrum_blaze,\
                                                  wavelength, NN_coeffs, wavelength_payne,\
                                                  p0_initial=None, RV_prefit=True, blaze_normalized=True,\
                                                  RV_array=RV_array, polynomial_order=2)

# print best parameters
# recale back
popt_best_copy = np.copy(popt_best)
popt_best[:4] = (popt_best[:4] + 0.5)*(x_max-x_min) + x_min
popt_best[0] = popt_best[0]*1000.
print("[Teff [K], logg, Fe/H, Alpha/Fe] = ",\
      int(popt_best[0]*1.)/1.,\
      int(popt_best[1]*100.)/100.,\
      int(popt_best[2]*100.)/100.,\
      int(popt_best[3]*100.)/100.)
print("vbroad [km/s] = ", int(popt_best[-2]*10.)/10.)
print("RV [km/s] = ", int(popt_best[-1]*1000.)/10.)
print("Chi square = ", chi_square)


Finding the best initial radial velocity
1 / 6
2 / 6
3 / 6
4 / 6
5 / 6
6 / 6
[Teff [K], logg, Fe/H, Alpha/Fe] =  4930.0 1.31 -0.37 -0.19
vbroad [km/s] =  1.7
RV [km/s] =  -40.3
Chi square =  4.44785394354732


In [7]:
RV_array = np.array([popt_best[-1]])
popt_best, model_spec_best, chi_square = fitting.fitting_mike(spectrum, spectrum_err, spectrum_blaze,\
                                                  wavelength, NN_coeffs, wavelength_payne,\
                                                  p0_initial=None, RV_prefit=False, blaze_normalized=True,\
                                                  RV_array=RV_array, polynomial_order=2)

# print best parameters
# recale back
popt_best_copy = np.copy(popt_best)
popt_best[:4] = (popt_best[:4] + 0.5)*(x_max-x_min) + x_min
popt_best[0] = popt_best[0]*1000.
popt_best[-1] = popt_best[-1]*100.
print("[Teff [K], logg, Fe/H, Alpha/Fe] = ",\
      int(popt_best[0]*1.)/1.,\
      int(popt_best[1]*100.)/100.,\
      int(popt_best[2]*100.)/100.,\
      int(popt_best[3]*100.)/100.)
print("vbroad [km/s] = ", int(popt_best[-2]*10.)/10.)
print("RV [km/s] = ", int(popt_best[-1]*10.)/10.)
print("Chi square = ", chi_square)


First fitting the blaze-normalized spectrum
1 / 1
[Teff [K], logg, Fe/H, Alpha/Fe] =  4539.0 1.3 -0.63 -0.19
vbroad [km/s] =  1.9
RV [km/s] =  -39.8
Chi square =  16.24853100747457


In [8]:
poly_initial = fitting.fit_continuum(spectrum, spectrum_err, wavelength, popt_best,\
                                     model_spec_best, polynomial_order=6, previous_polynomial_order=2)
print(poly_initial.shape)

(34, 7)


In [9]:
RV_array = np.array([popt_best_copy[-1]])
p0_initial = np.concatenate([popt_best_copy[:4], poly_initial.ravel(), popt_best_copy[-2:]])
print(p0_initial.shape)
popt_best, model_spec_best, chi_square = fitting.fitting_mike(spectrum, spectrum_err, spectrum_blaze,\
                                                  wavelength, NN_coeffs, wavelength_payne,\
                                                  p0_initial = p0_initial,\
                                                  RV_prefit=False, blaze_normalized=False,\
                                                  RV_array=RV_array, polynomial_order=6)


(244,)
1 / 1


In [10]:
# print best parameters
# recale back
popt_best_copy = np.copy(popt_best)
popt_best[:4] = (popt_best[:4] + 0.5)*(x_max-x_min) + x_min
popt_best[0] = popt_best[0]*1000.
popt_best[-1] = popt_best[-1]*100.
print("[Teff [K], logg, Fe/H, Alpha/Fe] = ",\
      int(popt_best[0]*1.)/1.,\
      int(popt_best[1]*100.)/100.,\
      int(popt_best[2]*100.)/100.,\
      int(popt_best[3]*100.)/100.)
print("vbroad [km/s] = ", int(popt_best[-2]*10.)/10.)
print("RV [km/s] = ", int(popt_best[-1]*10.)/10.)
print("Chi square = ", chi_square)


[Teff [K], logg, Fe/H, Alpha/Fe] =  4638.0 0.9 -0.51 -0.19
vbroad [km/s] =  2.0
RV [km/s] =  -39.7
Chi square =  7.291870183272961


> Plot the fits.

In [ ]:
smooth_length = 30

# make plot for individual order
for k in range(wavelength.shape[0]):
    
    # initiate the plot
    fig = plt.figure(figsize=[18,20]);

    # combine labels
    ax = fig.add_subplot(111)
 
    ax.spines['top'].set_color('none')
    ax.spines['bottom'].set_color('none')
    ax.spines['left'].set_color('none')
    ax.spines['right'].set_color('none')
    ax.tick_params(labelcolor='w', top='off', bottom='off', left='off', right='off')

#----------------------------------------------------------------------
    # write in different panel
    for i in range(5):
    
        # wavelength range
        wavelength_min = np.min(wavelength[k,:])-10.
        wavelength_max = np.max(wavelength[k,:])+10.
        wave_period = (wavelength_max-wavelength_min)/5.
    
        ax = fig.add_subplot(5,1,i+1)
        plt.xlim([wavelength_min+wave_period*(i),wavelength_min+wave_period*(i+1)])
    
        # plot spectrum
        plt.ylim([0.,2.])
        plt.plot(wavelength[k,:], spectrum[k,:], lw=2, label="MIKE", color=cb2[0])
        plt.plot(wavelength[k,:], model_spec_best[k,:], label="Kurucz", lw=2, color=cb2[1])
    
        # share also the measurement error
        plt.fill_between(wavelength[k,:], model_spec_best[k,:]-spectrum_err[k,:],\
                         model_spec_best[k,:]+spectrum_err[k,:], alpha=0.5, color=cb2[1])
    
#----------------------------------------------------------------------
        # show the masked telluric region
        for j in range(spectrum_blaze[k,:].size-smooth_length):
            if np.min(spectrum_blaze[k,j:j+smooth_length]) \
                        < 0.9*np.max(spectrum_blaze[k,j:j+smooth_length]):
                plt.fill_between([wavelength[k,j],wavelength[k,j+smooth_length]],\
                                 [-1.,-1.], [3.5,3.5], alpha=0.5, color="gray")
    
    # add axis and legend
    plt.xlabel("Wavelength [A]")
    plt.legend(loc="lower right", fontsize=28, frameon=False,\
                borderpad=0.05, labelspacing=0.1)

    # save figure
    plt.tight_layout()
    plt.savefig("Star2_Fit_" +str(k) + ".png")
    plt.close()

findfont: Font family ['Bitstream Vera Sans'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Bitstream Vera Sans'] not found. Falling back to DejaVu Sans.
